In [56]:
%pip install numpy
%pip install pandas

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [57]:
import pandas as pd
import numpy as np

ПОКАЗАТЕЛИ РЫНКА МЕТАЛЛА

In [3]:
df1 = pd.read_excel('Показатели рынка металла.xlsx')
df1

,dt,Производство арматуры Сортовой прокат_Арматура,Производство арматуры Сортовой прокат_Катанка,Производство арматуры Сортовой прокат_Прутки прочие,Производство сырья ЖРС_Агломерат,Производство сырья ЖРС_Аглоруда,Производство сырья ЖРС_Концентрат,Производство сырья ЖРС_Концентрат для дообогащения,Производство сырья ЖРС_Окатыши,Производство сырья Кокс_Кокс валовый,...,Экспорт сырья Лом_Лом стальной,Экспорт сырья Лом_Лом чугуна,Экспорт сырья Металлизованная продукция_Всего отгружено с предприятия,Экспорт сырья Металлизованная продукция_ГБЖ,Экспорт сырья Чугун_Всего отгружено с предприятия,Экспорт сырья Чугун_Литейный чугун,Экспорт сырья Чугун_Передельный чугун,Экспорт арматуры Сортовой прокат_Арматура,Экспорт арматуры Сортовой прокат_Катанка,Экспорт арматуры Сортовой прокат_Прутки прочие
0,2016-03-31,527,208,191,5127,376,9063,631,3691,2307,...,0,0,0,0,0,0,0,97,68,50
1,2016-04-30,563,231,216,4776,343,8850,608,3450,2297,...,0,0,0,0,0,0,0,118,79,55
2,2016-05-31,614,263,235,5107,420,9013,626,3849,2481,...,0,0,0,0,0,0,0,123,100,71
3,2016-06-30,659,242,237,4809,400,9275,613,3688,2372,...,0,0,0,0,0,0,0,129,78,61
4,2016-07-31,703,249,238,5147,420,9587,518,4032,2255,...,0,0,0,0,0,0,0,142,90,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2022-11-30,871,265,182,4433,502,8704,622,3885,2118,...,0,0,0,25,0,0,20,146,56,14
81,2022-12-31,928,253,248,4305,439,9171,655,4141,2034,...,0,0,0,22,0,0,36,94,72,14
82,2023-01-31,616,242,197,4296,532,8848,598,4292,2012,...,0,0,0,14,0,0,32,122,77,15
83,2023-02-28,699,286,189,4280,438,8750,607,4386,1985,...,0,0,0,20,0,10,27,141,124,13


In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 81 columns):
 #   Column                                                                 Non-Null Count  Dtype         
---  ------                                                                 --------------  -----         
 0   dt                                                                     85 non-null     datetime64[ns]
 1   Производство арматуры Сортовой прокат_Арматура                         85 non-null     int64         
 2   Производство арматуры Сортовой прокат_Катанка                          85 non-null     int64         
 3   Производство арматуры Сортовой прокат_Прутки прочие                    85 non-null     int64         
 4   Производство сырья ЖРС_Агломерат                                       85 non-null     int64         
 5   Производство сырья ЖРС_Аглоруда                                        85 non-null     int64         
 6   Производство сырья ЖРС_Концентрат   

In [5]:
# Проанализируем базовые статистики имеющихся признаков
df1.describe()

,dt,Производство арматуры Сортовой прокат_Арматура,Производство арматуры Сортовой прокат_Катанка,Производство арматуры Сортовой прокат_Прутки прочие,Производство сырья ЖРС_Агломерат,Производство сырья ЖРС_Аглоруда,Производство сырья ЖРС_Концентрат,Производство сырья ЖРС_Концентрат для дообогащения,Производство сырья ЖРС_Окатыши,Производство сырья Кокс_Кокс валовый,...,Экспорт сырья Лом_Лом стальной,Экспорт сырья Лом_Лом чугуна,Экспорт сырья Металлизованная продукция_Всего отгружено с предприятия,Экспорт сырья Металлизованная продукция_ГБЖ,Экспорт сырья Чугун_Всего отгружено с предприятия,Экспорт сырья Чугун_Литейный чугун,Экспорт сырья Чугун_Передельный чугун,Экспорт арматуры Сортовой прокат_Арматура,Экспорт арматуры Сортовой прокат_Катанка,Экспорт арматуры Сортовой прокат_Прутки прочие
count,85,85.000000,85.000000,85.00000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,...,85.000000,85.0,85.0,85.00000,85.0,85.000000,85.000000,85.000000,85.000000,85.000000
mean,2019-09-30 09:36:00,706.670588,264.988235,240.20000,4851.047059,424.458824,9599.611765,577.447059,4555.541176,2260.811765,...,0.517647,0.0,0.0,8.20000,0.0,0.835294,30.094118,128.047059,74.458824,57.035294
min,2016-03-31 00:00:00,491.000000,184.000000,176.00000,4201.000000,261.000000,8339.000000,427.000000,3450.000000,1936.000000,...,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.000000,78.000000,29.000000,13.000000
25%,2017-12-31 00:00:00,642.000000,242.000000,228.00000,4709.000000,385.000000,9198.000000,526.000000,4292.000000,2188.000000,...,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.000000,102.000000,62.000000,48.000000
50%,2019-09-30 00:00:00,698.000000,262.000000,243.00000,4855.000000,421.000000,9620.000000,584.000000,4709.000000,2283.000000,...,0.000000,0.0,0.0,0.00000,0.0,0.000000,15.000000,121.000000,73.000000,57.000000
75%,2021-06-30 00:00:00,787.000000,290.000000,257.00000,5020.000000,461.000000,10009.000000,626.000000,4866.000000,2334.000000,...,0.000000,0.0,0.0,20.00000,0.0,0.000000,36.000000,147.000000,83.000000,70.000000
max,2023-03-31 00:00:00,933.000000,379.000000,305.00000,5325.000000,558.000000,10787.000000,687.000000,5228.000000,2509.000000,...,18.000000,0.0,0.0,30.00000,0.0,10.000000,319.000000,213.000000,195.000000,113.000000
std,NaN,102.552703,39.175580,25.59241,239.827959,60.441147,529.619722,65.468914,423.062434,118.418195,...,2.514774,0.0,0.0,10.35995,0.0,2.429222,51.534460,31.789076,26.703198,18.960138


In [6]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df1.isna().sum())

Number of missing values in valid dataset:
dt                                                     0
Производство арматуры Сортовой прокат_Арматура         0
Производство арматуры Сортовой прокат_Катанка          0
Производство арматуры Сортовой прокат_Прутки прочие    0
Производство сырья ЖРС_Агломерат                       0
                                                      ..
Экспорт сырья Чугун_Литейный чугун                     0
Экспорт сырья Чугун_Передельный чугун                  0
Экспорт арматуры Сортовой прокат_Арматура              0
Экспорт арматуры Сортовой прокат_Катанка               0
Экспорт арматуры Сортовой прокат_Прутки прочие         0
Length: 81, dtype: int64


ГРУЗОПЕРЕВОЗКИ

In [7]:
df2 = pd.read_excel('Грузоперевозки.xlsx')
df2

,dt,Индекс стоимости грузоперевозок
0,2017-09-06,1000
1,2017-09-13,991
2,2017-09-20,992
3,2017-09-27,979
4,2017-10-04,985
...,...,...
297,2023-05-17,1434
298,2023-05-24,1452
299,2023-05-31,1487
300,2023-06-07,1500


In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   dt                               302 non-null    datetime64[ns]
 1   Индекс стоимости грузоперевозок  302 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 4.8 KB


In [9]:
# Проанализируем базовые статистики имеющихся признаков
df2.describe()

,dt,Индекс стоимости грузоперевозок
count,302,302.000000
mean,2020-07-25 12:00:00,1095.976821
min,2017-09-06 00:00:00,904.000000
25%,2019-02-14 18:00:00,981.250000
50%,2020-07-25 12:00:00,1022.500000
75%,2022-01-03 06:00:00,1235.750000
max,2023-06-14 00:00:00,1501.000000
std,NaN,153.428664


In [10]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df2.isna().sum())

Number of missing values in valid dataset:
dt                                 0
Индекс стоимости грузоперевозок    0
dtype: int64


МАКРОПОКАЗАТЕЛИ

In [11]:
df3 = pd.read_excel('Показатели рынка металла.xlsx')
df3

,dt,Производство арматуры Сортовой прокат_Арматура,Производство арматуры Сортовой прокат_Катанка,Производство арматуры Сортовой прокат_Прутки прочие,Производство сырья ЖРС_Агломерат,Производство сырья ЖРС_Аглоруда,Производство сырья ЖРС_Концентрат,Производство сырья ЖРС_Концентрат для дообогащения,Производство сырья ЖРС_Окатыши,Производство сырья Кокс_Кокс валовый,...,Экспорт сырья Лом_Лом стальной,Экспорт сырья Лом_Лом чугуна,Экспорт сырья Металлизованная продукция_Всего отгружено с предприятия,Экспорт сырья Металлизованная продукция_ГБЖ,Экспорт сырья Чугун_Всего отгружено с предприятия,Экспорт сырья Чугун_Литейный чугун,Экспорт сырья Чугун_Передельный чугун,Экспорт арматуры Сортовой прокат_Арматура,Экспорт арматуры Сортовой прокат_Катанка,Экспорт арматуры Сортовой прокат_Прутки прочие
0,2016-03-31,527,208,191,5127,376,9063,631,3691,2307,...,0,0,0,0,0,0,0,97,68,50
1,2016-04-30,563,231,216,4776,343,8850,608,3450,2297,...,0,0,0,0,0,0,0,118,79,55
2,2016-05-31,614,263,235,5107,420,9013,626,3849,2481,...,0,0,0,0,0,0,0,123,100,71
3,2016-06-30,659,242,237,4809,400,9275,613,3688,2372,...,0,0,0,0,0,0,0,129,78,61
4,2016-07-31,703,249,238,5147,420,9587,518,4032,2255,...,0,0,0,0,0,0,0,142,90,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2022-11-30,871,265,182,4433,502,8704,622,3885,2118,...,0,0,0,25,0,0,20,146,56,14
81,2022-12-31,928,253,248,4305,439,9171,655,4141,2034,...,0,0,0,22,0,0,36,94,72,14
82,2023-01-31,616,242,197,4296,532,8848,598,4292,2012,...,0,0,0,14,0,0,32,122,77,15
83,2023-02-28,699,286,189,4280,438,8750,607,4386,1985,...,0,0,0,20,0,10,27,141,124,13


In [12]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 81 columns):
 #   Column                                                                 Non-Null Count  Dtype         
---  ------                                                                 --------------  -----         
 0   dt                                                                     85 non-null     datetime64[ns]
 1   Производство арматуры Сортовой прокат_Арматура                         85 non-null     int64         
 2   Производство арматуры Сортовой прокат_Катанка                          85 non-null     int64         
 3   Производство арматуры Сортовой прокат_Прутки прочие                    85 non-null     int64         
 4   Производство сырья ЖРС_Агломерат                                       85 non-null     int64         
 5   Производство сырья ЖРС_Аглоруда                                        85 non-null     int64         
 6   Производство сырья ЖРС_Концентрат   

In [13]:
# Проанализируем базовые статистики имеющихся признаков
df3.describe()

,dt,Производство арматуры Сортовой прокат_Арматура,Производство арматуры Сортовой прокат_Катанка,Производство арматуры Сортовой прокат_Прутки прочие,Производство сырья ЖРС_Агломерат,Производство сырья ЖРС_Аглоруда,Производство сырья ЖРС_Концентрат,Производство сырья ЖРС_Концентрат для дообогащения,Производство сырья ЖРС_Окатыши,Производство сырья Кокс_Кокс валовый,...,Экспорт сырья Лом_Лом стальной,Экспорт сырья Лом_Лом чугуна,Экспорт сырья Металлизованная продукция_Всего отгружено с предприятия,Экспорт сырья Металлизованная продукция_ГБЖ,Экспорт сырья Чугун_Всего отгружено с предприятия,Экспорт сырья Чугун_Литейный чугун,Экспорт сырья Чугун_Передельный чугун,Экспорт арматуры Сортовой прокат_Арматура,Экспорт арматуры Сортовой прокат_Катанка,Экспорт арматуры Сортовой прокат_Прутки прочие
count,85,85.000000,85.000000,85.00000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,...,85.000000,85.0,85.0,85.00000,85.0,85.000000,85.000000,85.000000,85.000000,85.000000
mean,2019-09-30 09:36:00,706.670588,264.988235,240.20000,4851.047059,424.458824,9599.611765,577.447059,4555.541176,2260.811765,...,0.517647,0.0,0.0,8.20000,0.0,0.835294,30.094118,128.047059,74.458824,57.035294
min,2016-03-31 00:00:00,491.000000,184.000000,176.00000,4201.000000,261.000000,8339.000000,427.000000,3450.000000,1936.000000,...,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.000000,78.000000,29.000000,13.000000
25%,2017-12-31 00:00:00,642.000000,242.000000,228.00000,4709.000000,385.000000,9198.000000,526.000000,4292.000000,2188.000000,...,0.000000,0.0,0.0,0.00000,0.0,0.000000,0.000000,102.000000,62.000000,48.000000
50%,2019-09-30 00:00:00,698.000000,262.000000,243.00000,4855.000000,421.000000,9620.000000,584.000000,4709.000000,2283.000000,...,0.000000,0.0,0.0,0.00000,0.0,0.000000,15.000000,121.000000,73.000000,57.000000
75%,2021-06-30 00:00:00,787.000000,290.000000,257.00000,5020.000000,461.000000,10009.000000,626.000000,4866.000000,2334.000000,...,0.000000,0.0,0.0,20.00000,0.0,0.000000,36.000000,147.000000,83.000000,70.000000
max,2023-03-31 00:00:00,933.000000,379.000000,305.00000,5325.000000,558.000000,10787.000000,687.000000,5228.000000,2509.000000,...,18.000000,0.0,0.0,30.00000,0.0,10.000000,319.000000,213.000000,195.000000,113.000000
std,NaN,102.552703,39.175580,25.59241,239.827959,60.441147,529.619722,65.468914,423.062434,118.418195,...,2.514774,0.0,0.0,10.35995,0.0,2.429222,51.534460,31.789076,26.703198,18.960138


In [14]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df3.isna().sum())

Number of missing values in valid dataset:
dt                                                     0
Производство арматуры Сортовой прокат_Арматура         0
Производство арматуры Сортовой прокат_Катанка          0
Производство арматуры Сортовой прокат_Прутки прочие    0
Производство сырья ЖРС_Агломерат                       0
                                                      ..
Экспорт сырья Чугун_Литейный чугун                     0
Экспорт сырья Чугун_Передельный чугун                  0
Экспорт арматуры Сортовой прокат_Арматура              0
Экспорт арматуры Сортовой прокат_Катанка               0
Экспорт арматуры Сортовой прокат_Прутки прочие         0
Length: 81, dtype: int64


ТОПЛИВО

In [15]:
df4 = pd.read_excel('Топливо.xlsx')
df4

,dt,Цена на бензин,Цена на дт
0,2017-02-28,35.91,36.15
1,2017-03-31,36.11,36.58
2,2017-04-30,36.41,36.69
3,2017-05-31,36.64,36.80
4,2017-06-30,36.91,36.87
...,...,...,...
71,2023-01-31,51.65,56.73
72,2023-02-28,51.68,57.34
73,2023-03-31,51.67,57.33
74,2023-04-30,51.67,57.38


In [16]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   dt              76 non-null     datetime64[ns]
 1   Цена на бензин  76 non-null     float64       
 2   Цена на дт      76 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 1.9 KB


In [17]:
# Проанализируем базовые статистики имеющихся признаков
df4.describe()

,dt,Цена на бензин,Цена на дт
count,76,76.000000,76.000000
mean,2020-04-15 03:47:22.105263104,45.355000,46.559342
min,2017-02-28 00:00:00,35.910000,36.150000
25%,2018-09-22 12:00:00,43.825000,43.892500
50%,2020-04-15 00:00:00,45.125000,46.605000
75%,2021-11-07 12:00:00,49.680000,49.445000
max,2023-05-31 00:00:00,51.690000,57.390000
std,NaN,4.756944,5.770784


In [18]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df4.isna().sum())

Number of missing values in valid dataset:
dt                0
Цена на бензин    0
Цена на дт        0
dtype: int64


ЦЕНЫ НА СЫРЬЁ

In [19]:
df5 = pd.read_excel('Цены на сырье.xlsx')
df5

,dt,"Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС","Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т","Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС","ЖРС_Китай Iron ore fines Fe 62%, CFR","Чугун_CFR Италия, $/т","Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС","ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т",...,"Лом_3А, РФ FCA ж/д Свердловская обл, руб./т, без НДС","Чугун_CFR Турция, $/т","ЖРС_Средневзвешенная цена окатыши Fe 62-65,5%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена концентрат Fe 64-68%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена аглоруда Fe 52-60%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в аглоруде, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в концентрате, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в окатыши, Россия FCA руб./т, без НДС","ЖРС_Украинский концентрат Fe 65-68%, DAP граница Украины с Европой, $/т","ЖРС_Украинские окатыши Fe 65-66%, DAP граница Украины с Европой, $/т"
0,2015-02-09,3850,228,12344,10009,12728,62,NaN,3700.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-02-16,3850,206,12271,10432,12852,63,NaN,3700.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-02-23,3850,207,12645,11981,13034,64,NaN,3700.0,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-03-02,3850,214,12696,11912,12910,63,NaN,3700.0,72.0,...,NaN,NaN,3540.0,2097.0,1261.0,23.0,NaN,NaN,NaN,NaN
4,2015-03-09,4400,232,12916,12061,12964,61,NaN,4050.0,70.0,...,NaN,NaN,3540.0,2097.0,1261.0,23.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,2023-02-13,9500,399,25741,23774,24100,124,505.0,7500.0,136.5,...,22765.0,457.5,5043.0,4382.0,2577.0,50.0,66.0,78.0,116.0,154.0
419,2023-02-20,9500,390,25837,24060,24184,124,500.0,7500.0,137.5,...,23414.0,465.0,5043.0,4382.0,2577.0,50.0,66.0,78.0,116.0,154.0
420,2023-02-27,9500,411,25814,24275,24047,130,502.5,7500.0,143.0,...,24014.0,455.0,5547.0,4898.0,2921.0,56.0,74.0,85.0,122.0,159.0
421,2023-03-06,10000,422,25821,25347,24055,126,500.0,7500.0,139.0,...,24005.0,467.5,5547.0,4898.0,2921.0,56.0,74.0,85.0,122.0,159.0


In [20]:
df5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 423 entries, 0 to 422
Data columns (total 59 columns):
 #   Column                                                                                            Non-Null Count  Dtype         
---  ------                                                                                            --------------  -----         
 0   dt                                                                                                423 non-null    datetime64[ns]
 1   Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС                                  423 non-null    int64         
 2   Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т                                                          423 non-null    int64         
 3   Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС                                                      423 non-null    int64         
 4   Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС                                      

In [21]:
# Проанализируем базовые статистики имеющихся признаков
df5.describe()

,dt,"Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС","Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т","Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС","Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС","ЖРС_Китай Iron ore fines Fe 62%, CFR","Чугун_CFR Италия, $/т","Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС","ЖРС_Российские окатыши Fe 62-65,5%, SiO2 5,8-8,65, DAP Забайкальск-Манжули, $/т",...,"Лом_3А, РФ FCA ж/д Свердловская обл, руб./т, без НДС","Чугун_CFR Турция, $/т","ЖРС_Средневзвешенная цена окатыши Fe 62-65,5%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена концентрат Fe 64-68%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена аглоруда Fe 52-60%, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в аглоруде, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в концентрате, Россия FCA руб./т, без НДС","ЖРС_Средневзвешенная цена за 1 % Fe в окатыши, Россия FCA руб./т, без НДС","ЖРС_Украинский концентрат Fe 65-68%, DAP граница Украины с Европой, $/т","ЖРС_Украинские окатыши Fe 65-66%, DAP граница Украины с Европой, $/т"
count,423,423.000000,423.000000,423.000000,423.000000,423.000000,423.000000,318.000000,423.000000,423.000000,...,10.000000,318.000000,420.000000,420.000000,420.000000,420.000000,216.000000,216.000000,268.000000,268.000000
mean,2019-02-25 00:00:00,8615.898345,299.574468,18144.458629,16368.888889,17777.297872,93.075650,431.336478,7163.416076,113.814421,...,21682.500000,418.937107,6237.488095,4541.047619,2540.230952,44.338095,93.523148,133.078704,108.363806,149.309701
min,2015-02-09 00:00:00,3850.000000,153.000000,9103.000000,8016.000000,9052.000000,39.000000,285.500000,3700.000000,48.500000,...,18353.000000,280.000000,2456.000000,1751.000000,1049.000000,21.000000,51.000000,71.000000,72.000000,101.000000
25%,2017-02-16 12:00:00,5100.000000,241.000000,13531.000000,11803.000000,13218.000000,64.000000,355.000000,4400.000000,86.500000,...,19426.750000,345.000000,3742.000000,2740.000000,1863.000000,32.000000,63.000000,82.000000,81.000000,116.500000
50%,2019-02-25 00:00:00,7750.000000,277.000000,16714.000000,15567.000000,16698.000000,84.000000,386.250000,6450.000000,106.500000,...,22351.500000,375.000000,5035.000000,3889.000000,2470.000000,44.000000,80.500000,122.000000,98.250000,139.500000
75%,2021-03-04 12:00:00,9950.000000,335.000000,21056.000000,18977.500000,20097.500000,116.000000,501.125000,7950.000000,126.000000,...,23857.250000,475.000000,7693.250000,5274.000000,3119.000000,55.000000,111.000000,172.000000,118.000000,164.500000
max,2023-03-13 00:00:00,22750.000000,614.000000,35186.000000,31137.000000,34263.000000,225.000000,855.000000,19000.000000,260.000000,...,24573.000000,804.000000,17343.000000,12649.000000,6679.000000,97.000000,192.000000,278.000000,218.000000,268.000000
std,NaN,4488.050210,87.248968,6144.865688,5698.427962,5911.305094,38.819841,112.904764,3693.588271,44.895262,...,2435.623181,108.238145,3416.338491,2526.936288,1150.619087,18.102729,38.884078,57.009737,35.977692,39.725594


In [22]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df5.isna().sum())

Number of missing values in valid dataset:
dt                                                                                                    0
Концентрат коксующегося угля_Россия марка ОС FCA руб./т, без НДС                                      0
Лом_HMS 1/2 80:20, FOB EC Роттердам, $/т                                                              0
Лом_3А, РФ CPT ж/д Южный ФО, руб./т, без НДС                                                          0
Лом_3А, РФ CPT ж/д Сибирский ФО, руб./т, без НДС                                                      0
Лом_3А, РФ CPT ж/д Центральный ФО, руб./т, без НДС                                                    0
ЖРС_Китай Iron ore fines Fe 62%, CFR                                                                  0
Чугун_CFR Италия, $/т                                                                               105
Концентрат коксующегося угля_Россия марка КО FCA руб./т, без НДС                                      0
ЖРС_Российские окатыш

ИНДЕКС LME

In [23]:
df6 = pd.read_excel('Индекс LME.xlsx')
df6

,дата,цена
0,2021-02-12,607
1,2021-02-15,614
2,2021-02-16,615
3,2021-02-17,606
4,2021-02-18,604
...,...,...
512,2023-03-22,694.5
513,2023-03-23,680
514,2023-03-24,706.5
515,2023-03-27,725


In [24]:
df6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   дата    516 non-null    datetime64[ns]
 1   цена    517 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 8.2+ KB


In [25]:
# Проанализируем базовые статистики имеющихся признаков
df6.describe()

,дата
count,516
mean,2022-02-26 23:40:27.906976768
min,2021-02-12 00:00:00
25%,2021-08-17 18:00:00
50%,2022-02-19 12:00:00
75%,2022-09-22 06:00:00
max,2023-03-27 00:00:00


In [26]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df6.isna().sum())

Number of missing values in valid dataset:
дата    1
цена    0
dtype: int64


ДАННЫЕ РЫНКА СТРОЙМАТЕРИАЛОВ

In [27]:
df7 = pd.read_excel('Данные рынка стройматериалов.xlsx')
df7

,dt,Импорт цемента ЖД,Экспорт цемента ЖД,Отгрузка цемента хоппер ЖД,Отгрузка цемента остальные ЖД,Производство цемента,Импорт цемента,Экспорт цемента,Потребление цемента,Отгрузка цемента,Цена цемента с НДС,Цена цемента без НДС,Производство бетона РФ,Производство бетона МО,Производство бетона СПБ
0,2021-04-30,40.751000,27.530000,987.54000,201.757000,2.3891,52.331737,31.491150,2.290941,1.189297,3769.20,4968.10,1.919781,223.405,198.589
1,2021-05-31,69.979000,43.423000,1071.83200,256.733000,2.7115,79.639340,50.164150,2.670775,1.328565,3745.23,4822.46,2.059197,293.411,162.181
2,2021-06-30,105.525000,58.226000,1495.35100,446.953000,3.8468,125.590100,64.990750,3.984199,1.942304,3790.64,5299.51,2.837929,424.112,254.893
3,2021-07-31,125.093000,88.930000,1754.57600,573.133000,4.8559,182.176201,105.965900,4.933510,2.327709,3853.30,5031.96,3.302281,492.623,301.091
4,2021-08-31,124.438000,112.454000,1859.54900,659.929000,5.7240,193.681060,141.093150,5.762688,2.519478,3915.07,5240.72,3.788132,562.724,303.961
5,2021-09-30,125.669000,143.621000,1998.19100,813.518000,6.4010,217.010540,171.180400,6.674830,2.811709,3966.72,5163.41,4.296160,580.444,345.532
6,2021-10-31,122.030000,129.403000,1889.19600,989.297000,6.6908,232.288189,153.850060,6.881838,2.878493,4035.60,5426.48,5.773625,1082.764,373.798
7,2021-11-30,111.634000,120.585000,1844.53400,928.144000,6.6463,209.976386,142.032550,6.780344,2.772678,4097.58,5323.64,5.943797,1115.723,380.008
8,2021-12-31,97.011000,112.507000,1689.87300,789.147000,6.1945,178.660367,132.239750,6.386421,2.479020,4172.67,5397.43,5.965973,1090.623,361.167
9,2022-01-31,105.816000,102.736000,1778.07200,600.256000,6.0097,201.859649,119.818990,6.036641,2.378328,4231.15,5480.04,5.608766,1085.405,334.512


In [28]:
df7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   dt                             25 non-null     datetime64[ns]
 1   Импорт цемента ЖД              25 non-null     float64       
 2   Экспорт цемента ЖД             25 non-null     float64       
 3   Отгрузка цемента хоппер ЖД     25 non-null     float64       
 4   Отгрузка цемента остальные ЖД  25 non-null     float64       
 5   Производство цемента           25 non-null     float64       
 6   Импорт цемента                 25 non-null     float64       
 7   Экспорт цемента                25 non-null     float64       
 8   Потребление цемента            25 non-null     float64       
 9   Отгрузка цемента               25 non-null     float64       
 10  Цена цемента с НДС             25 non-null     float64       
 11  Цена цемента без НДС 

In [29]:
# Проанализируем базовые статистики имеющихся признаков
df7.describe()

,dt,Импорт цемента ЖД,Экспорт цемента ЖД,Отгрузка цемента хоппер ЖД,Отгрузка цемента остальные ЖД,Производство цемента,Импорт цемента,Экспорт цемента,Потребление цемента,Отгрузка цемента,Цена цемента с НДС,Цена цемента без НДС,Производство бетона РФ,Производство бетона МО,Производство бетона СПБ
count,25,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,2022-04-30 20:09:36,106.961964,76.048164,1569.077840,529.320857,4.938496,170.974773,89.989560,5.005069,2.098401,4458.290000,5915.457200,1.963231,843.746400,298.747400
min,2021-04-30 00:00:00,40.751000,27.530000,970.141880,193.233378,2.389100,52.331737,31.491150,2.290941,1.163375,3745.230000,4822.460000,0.002264,223.405000,162.181000
25%,2021-10-31 00:00:00,83.881918,56.712400,1280.205140,279.997752,3.707600,125.590100,62.101180,3.642894,1.553980,4035.600000,5299.510000,0.004034,580.444000,247.050000
50%,2022-04-30 00:00:00,105.816000,69.389800,1689.873000,535.570704,5.012800,177.904170,84.986894,4.995165,2.266877,4312.660000,5653.470000,0.006082,845.993000,318.986000
75%,2022-10-31 00:00:00,124.438000,88.930000,1844.534000,716.829805,6.201000,206.347401,105.965900,6.447753,2.555187,5013.970000,6687.320000,3.788132,1090.623000,349.822000
max,2023-04-30 00:00:00,182.082930,143.621000,1998.191000,989.297000,6.792200,280.405130,171.180400,6.938954,2.878493,5174.270000,6993.720000,5.965973,1367.746000,383.548000
std,NaN,33.071278,30.034960,332.002773,249.861484,1.487395,58.413138,37.172690,1.568539,0.567652,512.388713,755.819131,2.307018,339.176184,67.354162


In [30]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df7.isna().sum())

Number of missing values in valid dataset:
dt                               0
Импорт цемента ЖД                0
Экспорт цемента ЖД               0
Отгрузка цемента хоппер ЖД       0
Отгрузка цемента остальные ЖД    0
Производство цемента             0
Импорт цемента                   0
Экспорт цемента                  0
Потребление цемента              0
Отгрузка цемента                 0
Цена цемента с НДС               0
Цена цемента без НДС             0
Производство бетона РФ           0
Производство бетона МО           0
Производство бетона СПБ          0
dtype: int64


SAMPLE_SUBMISSION

In [31]:
df8 = pd.read_excel('sample_submission.xlsx')
df8

,dt,Цена на арматуру,Объем
0,2022-09-05,46000,2
1,2022-09-12,46000,0
2,2022-09-19,46000,2
3,2022-09-26,45500,0
4,2022-10-03,45500,2
5,2022-10-10,45000,0
6,2022-10-17,44000,2
7,2022-10-24,43000,0
8,2022-10-31,42000,2
9,2022-11-07,41500,0


In [32]:
df8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dt                28 non-null     datetime64[ns]
 1   Цена на арматуру  28 non-null     int64         
 2   Объем             28 non-null     int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 800.0 bytes


In [33]:
# Проанализируем базовые статистики имеющихся признаков
df8.describe()

,dt,Цена на арматуру,Объем
count,28,28.000000,28.00000
mean,2022-12-08 12:00:00,44250.000000,1.00000
min,2022-09-05 00:00:00,40000.000000,0.00000
25%,2022-10-22 06:00:00,41925.000000,0.00000
50%,2022-12-08 12:00:00,42900.000000,1.00000
75%,2023-01-24 18:00:00,45625.000000,2.00000
max,2023-03-13 00:00:00,55500.000000,2.00000
std,NaN,3645.951585,1.01835


In [34]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df8.isna().sum())

Number of missing values in valid dataset:
dt                  0
Цена на арматуру    0
Объем               0
dtype: int64


NMLK АКЦИИ

In [35]:
df9 = pd.read_csv('NLMK Акции.csv')
df9

,Date,Price,Open,High,Low,Vol.,Change %
0,03/31/2023,129.32,130.60,130.60,128.36,3.54M,-0.98%
1,03/30/2023,130.60,132.30,133.44,129.80,4.85M,-0.94%
2,03/29/2023,131.84,128.00,134.44,127.10,10.37M,3.37%
3,03/28/2023,127.54,128.58,128.94,126.56,2.27M,-0.50%
4,03/27/2023,128.18,127.00,128.42,126.86,2.81M,1.33%
...,...,...,...,...,...,...,...
1048,01/10/2019,154.12,153.52,154.12,152.26,1.96M,0.22%
1049,01/09/2019,153.78,150.36,154.46,150.08,4.03M,2.44%
1050,01/08/2019,150.12,150.98,151.34,149.64,3.12M,-3.57%
1051,01/04/2019,155.68,156.80,157.46,154.80,2.91M,-1.06%


In [36]:
df9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1053 non-null   object 
 1   Price     1053 non-null   float64
 2   Open      1053 non-null   float64
 3   High      1053 non-null   float64
 4   Low       1053 non-null   float64
 5   Vol.      1053 non-null   object 
 6   Change %  1053 non-null   object 
dtypes: float64(4), object(3)
memory usage: 57.7+ KB


In [37]:
# Проанализируем базовые статистики имеющихся признаков
df9.describe()

,Price,Open,High,Low
count,1053.000000,1053.000000,1053.000000,1053.000000
mean,167.054872,167.038784,169.227388,164.553238
std,46.859276,46.798293,47.358822,46.218152
min,73.900000,71.980000,78.000000,70.400000
25%,129.500000,129.440000,131.200000,127.700000
50%,156.000000,156.400000,157.940000,154.420000
75%,211.180000,211.500000,214.180000,207.620000
max,279.240000,280.180000,282.460000,276.220000


In [38]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df9.isna().sum())

Number of missing values in valid dataset:
Date        0
Price       0
Open        0
High        0
Low         0
Vol.        0
Change %    0
dtype: int64


MAGN АКЦИИ

In [39]:
df10 = pd.read_csv('MAGN Акции.csv')
df10

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %
0,31.03.2023,"39,69","40,21","40,24","39,52","18,23M","-1,38%"
1,30.03.2023,"40,24","40,60","41,00","40,02","27,14M","-0,87%"
2,29.03.2023,"40,60","39,15","40,65","38,96","48,70M","3,70%"
3,28.03.2023,"39,15","39,45","39,75","38,91","17,02M","-0,62%"
4,27.03.2023,"39,40","38,99","39,48","38,92","20,95M","1,26%"
...,...,...,...,...,...,...,...
1048,10.01.2019,"44,95","44,55","44,95","44,17","9,15M","0,33%"
1049,09.01.2019,"44,80","43,23","44,80","43,02","12,20M","3,80%"
1050,08.01.2019,"43,16","43,43","43,70","42,58","7,33M","-0,48%"
1051,04.01.2019,"43,37","42,98","43,62","42,80","5,64M","0,91%"


In [40]:
df10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Дата    1053 non-null   object
 1   Цена    1053 non-null   object
 2   Откр.   1053 non-null   object
 3   Макс.   1053 non-null   object
 4   Мин.    1053 non-null   object
 5   Объём   1053 non-null   object
 6   Изм. %  1053 non-null   object
dtypes: object(7)
memory usage: 57.7+ KB


In [41]:
# Проанализируем базовые статистики имеющихся признаков
df10.describe()

,Дата,Цена,Откр.,Макс.,Мин.,Объём,Изм. %
count,1053,1053,1053,1053,1053,1053,1053
unique,1053,861,840,865,848,920,559
top,31.03.2023,"38,10","40,00","39,95","38,31","3,59M","0,00%"
freq,1,6,10,5,6,4,7


In [42]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df10.isna().sum())

Number of missing values in valid dataset:
Дата      0
Цена      0
Откр.     0
Макс.     0
Мин.      0
Объём     0
Изм. %    0
dtype: int64


CHMF АКЦИИ

In [43]:
df11 = pd.read_csv('CHMF Акции.csv')
df11

,Date,Price,Open,High,Low,Vol.,Change %
0,03/31/2023,"1,043.20","1,061.00","1,061.00","1,042.00",571.72K,-1.68%
1,03/30/2023,"1,061.00","1,074.60","1,084.80","1,058.00",1.30M,-1.27%
2,03/29/2023,"1,074.60","1,040.60","1,078.40","1,035.40",1.79M,3.53%
3,03/28/2023,"1,038.00","1,041.40","1,044.00","1,033.00",269.09K,-0.17%
4,03/27/2023,"1,039.80","1,036.20","1,042.60","1,032.20",410.74K,0.50%
...,...,...,...,...,...,...,...
1049,01/10/2019,970.00,972.50,973.40,958.60,725.45K,-0.27%
1050,01/09/2019,972.60,953.10,973.50,950.00,1.03M,2.09%
1051,01/08/2019,952.70,952.00,960.00,941.30,565.63K,0.32%
1052,01/04/2019,949.70,934.00,949.70,931.10,335.53K,1.66%


In [44]:
df11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054 entries, 0 to 1053
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      1054 non-null   object
 1   Price     1054 non-null   object
 2   Open      1054 non-null   object
 3   High      1054 non-null   object
 4   Low       1054 non-null   object
 5   Vol.      1053 non-null   object
 6   Change %  1054 non-null   object
dtypes: object(7)
memory usage: 57.8+ KB


In [45]:
# Проанализируем базовые статистики имеющихся признаков
df11.describe()

,Date,Price,Open,High,Low,Vol.,Change %
count,1054,1054,1054,1054,1054,1053,1054
unique,1054,891,861,885,881,695,530
top,03/31/2023,"1,595.60",894.00,885.00,926.00,1.33M,1.21%
freq,1,4,4,5,4,12,8


In [46]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df11.isna().sum())

Number of missing values in valid dataset:
Date        0
Price       0
Open        0
High        0
Low         0
Vol.        1
Change %    0
dtype: int64


TEST

In [48]:
df12 = pd.read_excel('test.xlsx')
df12

,dt,Цена на арматуру
0,2022-09-05,46000
1,2022-09-12,46000
2,2022-09-19,46000
3,2022-09-26,45500
4,2022-10-03,45500
5,2022-10-10,45000
6,2022-10-17,44000
7,2022-10-24,43000
8,2022-10-31,42000
9,2022-11-07,41500


In [49]:
df12.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dt                28 non-null     datetime64[ns]
 1   Цена на арматуру  28 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 576.0 bytes


In [50]:
# Проанализируем базовые статистики имеющихся признаков
df12.describe()

,dt,Цена на арматуру
count,28,28.000000
mean,2022-12-08 12:00:00,44250.000000
min,2022-09-05 00:00:00,40000.000000
25%,2022-10-22 06:00:00,41925.000000
50%,2022-12-08 12:00:00,42900.000000
75%,2023-01-24 18:00:00,45625.000000
max,2023-03-13 00:00:00,55500.000000
std,NaN,3645.951585


In [51]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df12.isna().sum())

Number of missing values in valid dataset:
dt                  0
Цена на арматуру    0
dtype: int64


TRAIN

In [52]:
df13 = pd.read_excel('train.xlsx')
df13

,dt,Цена на арматуру
0,2015-01-05,32900
1,2015-01-12,32500
2,2015-01-19,32000
3,2015-01-26,30700
4,2015-02-02,30300
...,...,...
395,2022-08-01,47500
396,2022-08-08,49000
397,2022-08-15,47500
398,2022-08-22,46500


In [53]:
df13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dt                400 non-null    datetime64[ns]
 1   Цена на арматуру  400 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 6.4 KB


In [54]:
# Проанализируем базовые статистики имеющихся признаков
df13.describe()

,dt,Цена на арматуру
count,400,400.000000
mean,2018-11-01 12:00:00,39229.475000
min,2015-01-05 00:00:00,21000.000000
25%,2016-12-03 06:00:00,31075.000000
50%,2018-11-01 12:00:00,35450.000000
75%,2020-09-29 18:00:00,42000.000000
max,2022-08-29 00:00:00,79000.000000
std,NaN,13407.979668


In [55]:
# Смотрим на пропуски в данных
print("Number of missing values in valid dataset:")
print(df13.isna().sum())

Number of missing values in valid dataset:
dt                  0
Цена на арматуру    0
dtype: int64
